In [1]:
import numpy as np

from qibo.symbols import Z
from qibo import hamiltonians

from qiboml.models.encoding_decoding import *
from qiboml.models.ansatze import *

def run_model(model, x):
    for layer in model:
        x = layer.forward(x)
    return x

In [2]:
# binary data
model = [BinaryEncodingLayer(nqubits=3), ReuploadingLayer(nqubits=3, qubits=(0,2)), QuantumDecodingLayer(nqubits=3, qubits=(1,0))]
data = np.array([[0,0,0], [0,1,0], [1,1,0]])

for x in data:
    print(run_model(model, x).probabilities())


tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 1. 0. 0. 0. 0. 0.], shape=(8,), dtype=float64)
tf.Tensor([0. 0. 0. 0. 1. 0. 0. 0.], shape=(8,), dtype=float64)


In [3]:
# real data
nqubits = 4
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))
model = [PhaseEncodingLayer(nqubits, qubits=range(3)), ReuploadingLayer(nqubits), ExpectationLayer(nqubits, observable=observable, nshots=1000)]
data = np.pi * np.random.randn(3**2).reshape(3, 3)

for x in data:
    print(run_model(model, x))

[Qibo 0.2.10|INFO|2024-07-12 10:41:57]: Using qibojit (numba) backend on /CPU:0


0.5069999999999999
0.512
0.5209999999999999


In [4]:
"""
# pytorch interface
import torch
import qiboml.models.pytorch as pt

nqubits = 5
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))

model = torch.nn.Sequential(
    torch.nn.Linear(128,5),
    torch.nn.Sigmoid(),
    pt.BinaryEncodingLayer(5),
    pt.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
    pt.QuantumDecodingLayer(nqubits=5, qubits=reversed(range(5))),
    #pt.ExpectationLayer(nqubits=5, qubits=reversed(range(5)), observable=observable),
)

print(f"> Model: {model}")
print("> Parameters")
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.shape)

data = torch.randn(3, 128)
print("> Outputs")
for x in data:
    print(model(x).probabilities())
"""

'\n# pytorch interface\nimport torch\nimport qiboml.models.pytorch as pt\n\nnqubits = 5\nobservable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))\n\nmodel = torch.nn.Sequential(\n    torch.nn.Linear(128,5),\n    torch.nn.Sigmoid(),\n    pt.BinaryEncodingLayer(5),\n    pt.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),\n    pt.QuantumDecodingLayer(nqubits=5, qubits=reversed(range(5))),\n    #pt.ExpectationLayer(nqubits=5, qubits=reversed(range(5)), observable=observable),\n)\n\nprint(f"> Model: {model}")\nprint("> Parameters")\nfor name, param in model.named_parameters():\n    if param.requires_grad:\n        print(name, param.shape)\n\ndata = torch.randn(3, 128)\nprint("> Outputs")\nfor x in data:\n    print(model(x).probabilities())\n'

In [5]:
import torch
import qiboml.models.pytorch as pt
import qiboml.models.encoding_decoding as ed
import qiboml.models.ansatze as ans

q_model = pt.QuantumModel(
    layers = [
        ed.BinaryEncodingLayer(5),
        ans.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
        ed.ExpectationLayer(nqubits=5, qubits=reversed(range(5)), observable=observable, nshots=1000),
    ]
)
model = torch.nn.Sequential(
    torch.nn.Linear(128,5),
    torch.nn.Hardshrink(),
    q_model,
    torch.nn.Tanh()
)

for x in torch.randn(3,128):
    print(model(x))

tensor(0.7616, dtype=torch.float64)
tensor(0.7616, dtype=torch.float64)
tensor(0.7616, dtype=torch.float64)


In [6]:
# keras/tensorflow interface
import keras
import tensorflow as tf
import qiboml.models.keras as ks
import qiboml.models.encoding_decoding as ed
import qiboml.models.ansatze as ans

nqubits = 5
observable = hamiltonians.SymbolicHamiltonian(np.prod([Z(q) for q in range(nqubits)]))

q_model = ks.QuantumModel(
    layers = [
        ed.PhaseEncodingLayer(5),
        ans.ReuploadingLayer(nqubits=5, qubits=(0,2,4)),
        ed.ExpectationLayer(nqubits=5, qubits=reversed(range(5)), observable=observable, nshots=1000)
    ]
)

model = keras.Sequential([
    keras.layers.Dense(5, activation="sigmoid", name="dense"),
    q_model,
])


print(f"> Model: {model}")

data = tf.random.uniform((3,128))
print("> Outputs")
for x in data:
    x = tf.expand_dims(x, axis=0)
    print(model(x))



> Model: <Sequential name=sequential, built=False>
> Outputs
0.4252
0.39999999999999997
0.39640000000000003
